In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import joblib
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import shutil

In [2]:
all_images =  "E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\ALL-DEFECT-IMAGES"
target_dir = Path(r"E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\PNG-PARENT")
excel_path = "E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\XL-structural-defect\ServiceOrderItemDetails.xlsx"
sel_cat = ["boxing","swimming"]
temp_cat = ["Windows-sliding door"]
png_file_dir = "E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\PNG-FILES"

In [3]:
class ImageSeparator:
    def __init__(self,images_dir,target_dir,excel_path):
        self.excel_path = excel_path
        self.images_dir = images_dir
        self.df = pd.read_excel(self.excel_path,sheet_name="Sheet1") 
        self.target_dir = target_dir
      
    def getFolderNameFromXL(self):
        categories = self.df['Category']
        unique_categories = []
        for i in categories:
            if i not in unique_categories:
                unique_categories.append(i)
        return unique_categories
    
    
    def folder_creation(self,parent_dir):
        #try:
        categories = self.getFolderNameFromXL()

        for category in categories:
            folder = category.replace("/","-").replace(" ","_")
            path = os.path.join(parent_dir,folder)
            os.mkdir(path)
            print(f"folder {folder} created")
        target_folders = os.listdir(parent_dir)
        return target_folders
#         except:
#             print("oops folder creation is unsuccessful")
            
    def append_files_based_on_category(self):
        #try:
        target_folders = os.listdir(self.target_dir)
        category = [category for category in self.df["Category"]]
        file_names = os.listdir(self.images_dir)

        for file in tqdm(file_names):
            head, tail = os.path.splitext(file)

            if tail in [".jpg", ".jpe", ".jpeg",".png"]:
                v = head.split("_")
                snum = int(v[0])
                onum = int(float(v[1]))
                cat = self.df[(self.df["Service Order Number"]==snum) & (self.df["Item No. Order Doc."]==onum)]['Category'].values
                for folder_name in target_folders:
                    print(folder_name)
                    old_name = folder_name.replace("-","/").replace("_"," ")
                    if (cat == old_name) :
                        print(cat)
                        shutil.copy(f"{self.images_dir}\{file}",f"{self.target_dir}\{folder_name}\{file}")

        print("********* all files has been classified **********")   
#         except:
#             print("oops... flailed to save files")
            
                
# image_separator = ImageSeparator(all_images,target_dir,excel_path)
# image_separator.append_files_based_on_category()
    

In [7]:

class PrepareDataForModel:
    def __init__(self,augmented=True):
        self.size = (260,260)
        self.augmented = augmented
        
        
    def set_resolution(modelName,imgPath):
        path = "E:\STRUCTURAL-DEFECT-DETECTION\Super-Resolution -Models\ESPCN_x3.pb"
        img = cv2.imread(imgPath)
        sr_model = cv2.dnn_superres.DnnSuperResImpl_create()
        sr_model.readModel(path)
        model.setModel("espcn",3)
        sr_image = sr_model.upsample(img)
        img = sr_image[:,:,::-1]
        return img
        
    
    def data_augmentation(self,parent_dir_path,aug_X,category):
        data = np.array(aug_X)
        print(data[0][0].shape)
        #save_path = r"E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\op"
        save_path = f'{os.path.join(parent_dir_path,category)}'
        dataGenerator = ImageDataGenerator(samplewise_center=True,
                                           rotation_range=40,vertical_flip=True,horizontal_flip=True,
                                           brightness_range=(0.5,1.5),width_shift_range= (0.5,1.5),
                                           height_shift_range=(0.5,1.5),rescale=1./255.) 
        print(f"file saving to{save_path}")
        epoch = 15
        for batch in tqdm(dataGenerator.flow(data,batch_size=16,save_to_dir=save_path,
                                             save_prefix='aug',save_format='jpe')):
            epoch -=1
            if epoch <=1:
                break
    
        
    def set_image_size(self,file_name,parent_dir_path,category):
        path = os.path.join(os.path.join(parent_dir_path,category),file_name)
        head, tail = os.path.splitext(file_name)
        if tail in [".jpe"]:
            file = cv2.imread(path)
            file = cv2.resize(file,self.size)
            return file
              
        
    def data_augmentation_driver(self,parent_dir_path):
        X =[]
        y = []
        print('data_augmentation_driver initiated....')
        parent_dir = os.listdir(parent_dir_path)
        for category in tqdm(parent_dir):
            if category in sel_cat:
                aug_X = []
                sub_dir = os.listdir(os.path.join(parent_dir_path,category))
                for file_name in sub_dir:
                    path = os.path.join(os.path.join(parent_dir_path,category),file_name)
                    print(path)

                    head, tail = os.path.splitext(file_name)
                    if tail in [".jpg"]:
                        #print(file_name,category)
                        file = cv2.imread(path)
                        file = cv2.cvtColor(file,cv2.COLOR_BGR2RGB)
                        file = cv2.resize(file,self.size)
                    if not self.augmented:
                        aug_X.append(file)
                    else:
                        X.extend([file])
                        y.extend([category])
                if len(aug_X):
                    print("...data augmentation started...")
                    self.data_augmentation(parent_dir_path,aug_X,category)

        if self.augmented:
            self.augmented = False
            print("data_augmentation_driver session ended....")
            print("features and labels are ready")
            X = np.array(X)
            y = np.array(y)
            return X,y
        self.augmented = True
        if self.augmented:
            return (self.data_augmentation_driver(parent_dir_path))
            
        print('************Anugmentation successfull **************')
    
    def save_features(self,X,y):
        path = r"C:\Users\sanketh\Desktop\JUPYTER\DEEP-LEARNING\test-cats\new_X.pickle"
        joblib.dump(X,path)
        path = r"C:\Users\sanketh\Desktop\JUPYTER\DEEP-LEARNING\test-cats\new_y.pickle"
        joblib.dump(y,path)

    def label_encoder(self,y):
        encoder = LabelEncoder()
        y = encoder.fit_transform(y)
        print("Values has been encoded")
        return y
    

In [8]:
am=PrepareDataForModel()
new_x,new_y = am.data_augmentation_driver(r"E:\DEEP-LEARNING\video-classification\images")

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

data_augmentation_driver initiated....
E:\DEEP-LEARNING\video-classification\images\boxing\00000000.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000001.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000003.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000004.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000005.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000006.png
E:\DEEP-LEARNING\video-classification\images\boxing\00000007.png
E:\DEEP-LEARNING\video-classification\images\boxing\00000009.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000011.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000012.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000013.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000014.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000015.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000016.jpg
E:\DEEP-LEARNING\video-classification\images\boxing

E:\DEEP-LEARNING\video-classification\images\boxing\00000148.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000149.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000150.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000152.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000153.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000154.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000155.jpeg
E:\DEEP-LEARNING\video-classification\images\boxing\00000156.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000157.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000158.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000159.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000160.jpeg
E:\DEEP-LEARNING\video-classification\images\boxing\00000161.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000162.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000163.jpg
E:\DEEP-LEARNING\video-

E:\DEEP-LEARNING\video-classification\images\boxing\00000308.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000309.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000310.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000312.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000313.JPG
E:\DEEP-LEARNING\video-classification\images\boxing\00000314.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000315.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000316.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000317.jpeg
E:\DEEP-LEARNING\video-classification\images\boxing\00000318.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000319.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000320.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000321.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000322.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000324.jpg
E:\DEEP-LEARNING\video-c

E:\DEEP-LEARNING\video-classification\images\boxing\00000458.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000459.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000460.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000461.png
E:\DEEP-LEARNING\video-classification\images\boxing\00000462.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000463.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000464.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000465.jpeg
E:\DEEP-LEARNING\video-classification\images\boxing\00000466.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000467.jpeg
E:\DEEP-LEARNING\video-classification\images\boxing\00000468.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000469.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000470.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000471.jpg
E:\DEEP-LEARNING\video-classification\images\boxing\00000472.jpg
E:\DEEP-LEARNING\video-

...

E:\DEEP-LEARNING\video-classification\images\swimming\00000283.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000284.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000285.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000286.gif
E:\DEEP-LEARNING\video-classification\images\swimming\00000287.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000290.png
E:\DEEP-LEARNING\video-classification\images\swimming\00000291.JPG
E:\DEEP-LEARNING\video-classification\images\swimming\00000292.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000293.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000294.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000295.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000296.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000297.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000298.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000299

E:\DEEP-LEARNING\video-classification\images\swimming\00000431.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000432.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000433.jpeg
E:\DEEP-LEARNING\video-classification\images\swimming\00000434.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000435.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000436.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000437.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000438.png
E:\DEEP-LEARNING\video-classification\images\swimming\00000439.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000440.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000441.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000442.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000443.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000444.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\0000044

E:\DEEP-LEARNING\video-classification\images\swimming\00000577.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000578.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000579.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000580.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000581.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000582.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000583.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000584.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000585.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000586.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000587.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000588.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000589.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000590.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000591

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.28s/it]

E:\DEEP-LEARNING\video-classification\images\swimming\00000699.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000700.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000701.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000702.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000703.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000704.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000705.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000706.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000707.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000708.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000709.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000710.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000712.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000714.jpg
E:\DEEP-LEARNING\video-classification\images\swimming\00000715

In [9]:
y

array(['boxing', 'boxing', 'boxing', ..., 'swimming', 'swimming',
       'swimming'], dtype='<U8')

In [33]:
y = am.label_encoder(y)

Values has been encoded


In [10]:
am.save_features(new_x,new_y)

In [160]:
path = "E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\Structural-Defect-Images-Categories\Other"
def test_external(path):
    other_list = []
    others = os.listdir(path)
    count = 0
    for img in others:
        img_path = os.path.join(path,img)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(240,240))
        img = img/255.
        print(img.shape)
        other_list.extend([img])
        if(count > 20):
            break
        count +=1
    return other_list
        
        

## Adding column to excel


In [43]:
all_images = "E:\STRUCTURAL-DEFECT-DETECTION\STRUCTURAL-DEFECT-IMAGES\ALL-DEFECT-IMAGES-BACKUP"
df = pd.read_excel(excel_path,sheet_name="Sheet1")

def add_column(excel_path,images_path,df):
    df = pd.read_excel(excel_path,sheet_name="Sheet1")
    count = 0
    for file in tqdm(file_names):
                head, tail = os.path.splitext(file)
                name = f'{count}{tail}'
                new_name = os.path.join(all_images,name)
                old_name =os.path.join(all_images,file) 
                if tail in [".jpg", ".jpe", ".jpeg",".png"]:
                    v = head.split("_")
                    snum = int(v[0])
                    onum = int(float(v[1]))
                    index = df.index[(df["Service Order Number"]==snum) & (df["Item No. Order Doc."]==onum)][0]
                    df.iloc[index,0].fillna(value=0, inplace=True)
                    os.rename(old_name,new_name)
                    count +=1
    

In [96]:
import pandas as pd
df = pd.read_excel(excel_path,sheet_name="Sheet1")
df[(df["Service Order Number"]==8000086403) & (df["Item No. Order Doc."]==10)]["Defect Note 1"].values


array(['AC FILTER ACCESS - Unable to locate air conditioning filter'],
      dtype=object)

In [5]:
folder = Path(r"E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike")
for filename in (os.listdir(folder)):
    dst = f" {(filename)}.png"
    new = os.path.join(os.path.join(folder,dst))
    old = os.path.join(os.path.join(folder,filename))
    print(old)
    print(new)
    os.rename(old,new)

E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images127.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images127.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images128.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images128.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images129.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images129.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images130.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images130.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images131.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images131.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\images132.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ images132.jpg.png
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\indian-chief-bobber-dark-horse.jpg
E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\data\val\bike\ indian-chief-bobber-dark

In [2]:
path = r'E:\DEEP-LEARNING\YOLO\MASK\multi-cat'
file_names = os.listdir(path)
tpath = r'E:\DEEP-LEARNING\YOLO\MASK\copy'
for file in tqdm(file_names):
            head, tail = os.path.splitext(file)

            if tail in [".jpg"]:
                 shutil.copy(f"{path}\{file}",f"{tpath}\{file}")


100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 123.36it/s]


In [5]:
path = r"E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\multi_cat"
tpath = r"E:\DEEP-LEARNING\YOLO\Bike-Car-Yolo-V4\resize"
files = os.listdir(path)
for file in files:
    head,tail = os.path.splitext(file)
    if tail in [".png"]:
        img = cv2.imread(os.path.join(path,file))
        img = cv2.resize(img,(240,240))
        cv2.imwrite(os.path.join(tpath,file),img)
        

In [ ]:
path = r"E:\DEEP-LEARNING\YOLO\MASK\multi-cat"
files = os.listdir(path)
for file in tqdm(files):
    head, tail = os.path.splitext(file)
    if tail in [".jpg"]:

In [36]:
folder = r"E:\STRUCTURAL-DEFECT-DETECTION\TEST-DEFECT-DETECTION\painting\images"
files = os.listdir(folder)
count = 0
for file in files:
    head,tail = os.path.splitext(file)
    if tail not in [".jpg"]:
        dst = f" {(file)}.jpg"
        new = os.path.join(os.path.join(folder,dst))
        old = os.path.join(os.path.join(folder,file))
        os.rename(old,new)
        

In [35]:
folder = r"E:\STRUCTURAL-DEFECT-DETECTION\TEST-DEFECT-DETECTION\plumbing\Defect-Bound_Box\plumbing\yolo-format"
files = os.listdir(folder)
count = 0
for file in files:
    head,tail = os.path.splitext(file)
    name = f"plumbing-{count}.txt"
    new = os.path.join(os.path.join(folder,name))
    old = os.path.join(os.path.join(folder,file))
    os.rename(old,new)
    count+=1
   

In [38]:
folder = r"E:\STRUCTURAL-DEFECT-DETECTION\TEST-DEFECT-DETECTION\Bricklaying\brick-cat"
files = os.listdir(folder)
count = 0
for file in files:
    head,tail = os.path.splitext(file)
#     unwanted = head[-8:]
#     if ".jpe.png" in unwanted:
    print(file)

-old-red-brick-mold-covered-wall-the-surface-of-the-old-red-brick-1629278083.jpg
03fca2c3fb77d96088a68a052b685fab.jpg
0a8f7a0eb6d618090b8019c85f3328ea.jpg
10124611553_e532cb14b0_b.jpg
13310214825_1b78ab9dae_n.jpg
139364-592d8d566678d_img0601.jpg
14622665214_d11065c259.jpg
156719-59c123aa5bea7_cf8915c5-054a-482d-81e4-1c0e46b26868.jpg
169737669-texture-of-an-old-damaged-brick-wall-with-a-big-crack-in-it-as-a-vertical-background.jpg
200307282-001_XS.jpg
2017-07-27-12.b.jpg
240_F_219088579_KiWFjbjzOgfzdTTAA0nPU2ri1GshQkdb.jpg
360_F_163171838_XrS18tSEKA1uc4cutLdy29oJDFxBsvDF.jpg
360_F_28830296_MyGLN5p75o5HK09qrEoblpcHGOXcq2Q3.jpg
360_F_308745566_eEv5cxG0s8rhnCh2XReyS45GJ2tgBUDg.jpg
360_F_326802777_uO8hWRwWQME8bKlB63dLWOZosPaGW5YG.jpg
4718800725_5ce56b61ef_b.jpg
530f8a7150e27f2d7de724c1e4ed7bef.jpg
5693754541_ca31078388.jpg
6594037849_c778ed609a.jpg
7516e1d2e8bb6c2057e90f0f1983e31e.jpg
766-ad-sealing-weep-holes-brick-walls.jpg
81AEE7A9B4E381A8E383ACE383B3E382ACE381AEE5A381E381AEE3819DE381B0E

In [53]:
img_folder = r"E:\STRUCTURAL-DEFECT-DETECTION\TEST-DEFECT-DETECTION\Bricklaying\rash\image"
txt_folder = r"E:\STRUCTURAL-DEFECT-DETECTION\TEST-DEFECT-DETECTION\Bricklaying\rash\yolo"
img_files = os.listdir(img_folder)
txt_files = os.listdir(txt_folder)
counter = 1
for img,txt in zip(img_files,txt_files):
    im_head,im_tail = os.path.splitext(img)
    tx_head,tx_tail = os.path.splitext(txt)
    if im_head == tx_head:
        new_img =f"brick-{counter}.jpg" 
        new_txt = f"brick-{counter}.txt"
        old_im_name = os.path.join(img_folder,img)
        old_txt_name = os.path.join(txt_folder,txt)
        new_im_name = os.path.join(img_folder,new_img)
        new_txt_name = os.path.join(txt_folder,new_txt)
        os.rename(old_im_name,new_im_name)
        os.rename(old_txt_name,new_txt_name)
        counter +=1 

In [54]:
dic = {'san':1}

In [58]:
dic["san"]

1

In [59]:
class one:
    def __init__(self):
        pass
    def test(self):
        a = 1
        return a
    def temp(self):
        a = self.test()
        print(a)
        

In [60]:
o = one()

In [61]:
o.temp()

1


In [130]:
with open("test_file.txt","w") as file:
    file.write("classes = 3\n")
    file.write("filter = 14\n")


In [140]:
with open("test_file.txt") as file:
    print(file.read())

classes = 0
filter = 3


In [141]:
with open("test_file.txt","r") as file:
    line = file.readlines()
    print(line)
    line[0] = "classes = 10\n"
    file = open("test_file.txt", "w")
    file.writelines(line)
    file.close()
    

['classes = 0\n', 'filter = 3']


In [1]:
import cv2 as cv
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    print(frame)
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

[[[ 87  90 106]
  [ 87  90 105]
  [ 88  92 105]
  ...
  [ 62  55  80]
  [ 68  55  81]
  [ 70  56  83]]

 [[ 88  91 107]
  [ 88  91 107]
  [ 89  92 108]
  ...
  [ 62  56  80]
  [ 67  57  83]
  [ 69  58  84]]

 [[ 89  92 108]
  [ 89  92 108]
  [ 89  92 108]
  ...
  [ 62  58  81]
  [ 65  57  83]
  [ 68  59  85]]

 ...

 [[175 162 189]
  [175 162 189]
  [175 162 189]
  ...
  [150 149 180]
  [149 148 180]
  [149 148 180]]

 [[173 163 189]
  [173 163 189]
  [173 163 189]
  ...
  [150 149 180]
  [149 148 180]
  [149 148 180]]

 [[173 163 189]
  [173 163 189]
  [173 163 189]
  ...
  [150 149 180]
  [149 148 180]
  [150 149 181]]]
[[[ 80  93  99]
  [ 84  94  99]
  [ 86  94 100]
  ...
  [ 64  60  62]
  [ 66  63  66]
  [ 66  63  66]]

 [[ 80  93  99]
  [ 83  93  98]
  [ 85  92  99]
  ...
  [ 64  59  63]
  [ 65  62  65]
  [ 66  63  66]]

 [[ 85  92  99]
  [ 85  92  99]
  [ 85  92  99]
  ...
  [ 65  60  65]
  [ 64  60  65]
  [ 64  60  65]]

 ...

 [[168 162 170]
  [170 163 171]
  [168 162 170]
  ..

[[[ 88  88  81]
  [ 87  85  79]
  [ 87  83  79]
  ...
  [ 52  60  64]
  [ 51  59  63]
  [ 49  57  61]]

 [[ 88  88  81]
  [ 87  86  80]
  [ 86  85  80]
  ...
  [ 51  59  62]
  [ 51  60  61]
  [ 50  59  60]]

 [[ 88  88  81]
  [ 87  86  80]
  [ 87  86  80]
  ...
  [ 48  59  60]
  [ 47  60  61]
  [ 45  59  60]]

 ...

 [[162 158 149]
  [161 156 149]
  [162 157 151]
  ...
  [140 143 136]
  [142 144 137]
  [143 145 138]]

 [[162 157 151]
  [162 157 151]
  [164 158 152]
  ...
  [140 143 134]
  [142 144 135]
  [145 147 137]]

 [[164 158 152]
  [164 158 152]
  [164 158 152]
  ...
  [141 143 134]
  [142 144 135]
  [146 148 138]]]
[[[ 85  92  84]
  [ 84  90  83]
  [ 84  91  83]
  ...
  [ 54  58  65]
  [ 56  59  66]
  [ 56  59  66]]

 [[ 85  92  84]
  [ 84  91  83]
  [ 83  89  82]
  ...
  [ 55  59  61]
  [ 55  59  61]
  [ 56  60  62]]

 [[ 84  91  83]
  [ 83  89  82]
  [ 82  88  81]
  ...
  [ 55  60  59]
  [ 55  60  59]
  [ 55  60  60]]

 ...

 [[161 158 152]
  [161 158 152]
  [161 158 152]
  ..

[[[ 89  89  69]
  [ 89  88  67]
  [ 89  87  67]
  ...
  [ 49  63  61]
  [ 47  64  58]
  [ 46  63  57]]

 [[ 87  89  72]
  [ 88  88  72]
  [ 88  87  71]
  ...
  [ 46  61  57]
  [ 45  62  56]
  [ 44  62  56]]

 [[ 84  87  75]
  [ 87  88  75]
  [ 89  88  74]
  ...
  [ 44  60  53]
  [ 42  60  52]
  [ 42  60  52]]

 ...

 [[164 158 154]
  [162 157 154]
  [160 156 154]
  ...
  [140 146 141]
  [140 146 141]
  [139 145 140]]

 [[164 158 154]
  [162 157 155]
  [160 156 156]
  ...
  [144 146 141]
  [144 145 141]
  [145 145 141]]

 [[161 158 154]
  [160 156 153]
  [160 156 154]
  ...
  [141 144 141]
  [144 144 144]
  [145 144 145]]]
[[[ 86  91  81]
  [ 83  87  75]
  [ 84  87  73]
  ...
  [ 67  60  67]
  [ 68  60  65]
  [ 67  59  64]]

 [[ 85  89  82]
  [ 84  87  79]
  [ 85  87  78]
  ...
  [ 66  59  63]
  [ 67  59  62]
  [ 66  58  61]]

 [[ 81  87  80]
  [ 82  88  80]
  [ 82  89  79]
  ...
  [ 63  58  59]
  [ 62  58  58]
  [ 61  57  57]]

 ...

 [[166 156 159]
  [166 156 159]
  [166 156 159]
  ..

[[[ 71  88  75]
  [ 70  89  77]
  [ 72  92  81]
  ...
  [ 62  56  63]
  [ 64  57  65]
  [ 65  59  66]]

 [[ 74  89  76]
  [ 73  89  77]
  [ 73  90  79]
  ...
  [ 60  57  62]
  [ 59  58  63]
  [ 59  58  63]]

 [[ 78  92  80]
  [ 73  89  77]
  [ 72  88  78]
  ...
  [ 56  58  60]
  [ 54  58  60]
  [ 55  59  61]]

 ...

 [[164 158 154]
  [164 158 154]
  [164 158 154]
  ...
  [142 144 140]
  [142 146 141]
  [143 147 142]]

 [[164 158 154]
  [164 158 154]
  [164 158 154]
  ...
  [142 145 142]
  [141 147 142]
  [141 147 142]]

 [[165 159 155]
  [164 157 156]
  [162 155 156]
  ...
  [143 146 146]
  [141 146 144]
  [142 148 145]]]
[[[ 81  91  88]
  [ 78  90  88]
  [ 73  87  86]
  ...
  [ 39  60  63]
  [ 35  60  60]
  [ 33  59  59]]

 [[ 80  90  85]
  [ 78  87  84]
  [ 77  86  85]
  ...
  [ 43  60  62]
  [ 41  60  58]
  [ 39  58  56]]

 [[ 82  90  85]
  [ 79  87  84]
  [ 77  86  85]
  ...
  [ 44  59  61]
  [ 40  56  55]
  [ 39  55  53]]

 ...

 [[166 158 152]
  [165 158 152]
  [164 158 152]
  ..

[[[ 84  85  80]
  [ 81  84  79]
  [ 80  85  80]
  ...
  [ 34  60  56]
  [ 39  61  55]
  [ 41  62  56]]

 [[ 82  85  80]
  [ 82  85  80]
  [ 82  85  80]
  ...
  [ 36  58  55]
  [ 39  60  58]
  [ 41  61  60]]

 [[ 80  86  78]
  [ 81  87  80]
  [ 81  87  80]
  ...
  [ 39  58  55]
  [ 41  60  58]
  [ 41  60  58]]

 ...

 [[161 155 151]
  [162 156 154]
  [162 155 156]
  ...
  [143 145 137]
  [142 144 137]
  [142 144 137]]

 [[161 154 155]
  [162 155 156]
  [162 155 156]
  ...
  [141 143 134]
  [141 143 134]
  [142 144 135]]

 [[160 156 156]
  [160 156 156]
  [161 157 157]
  ...
  [135 142 133]
  [134 141 131]
  [137 143 134]]]
[[[ 80  91  81]
  [ 76  88  78]
  [ 77  88  78]
  ...
  [ 47  60  71]
  [ 48  61  74]
  [ 48  61  74]]

 [[ 82  91  81]
  [ 79  88  78]
  [ 79  88  78]
  ...
  [ 47  61  68]
  [ 46  60  68]
  [ 46  60  68]]

 [[ 82  91  81]
  [ 79  88  78]
  [ 79  88  78]
  ...
  [ 47  61  64]
  [ 45  60  63]
  [ 43  59  62]]

 ...

 [[165 159 155]
  [166 160 156]
  [165 159 155]
  ..

...

[[[ 87  92  87]
  [ 88  93  89]
  [ 87  92  87]
  ...
  [ 53  63  54]
  [ 52  60  53]
  [ 51  59  52]]

 [[ 87  92  87]
  [ 88  93  89]
  [ 87  92  87]
  ...
  [ 52  63  55]
  [ 53  61  56]
  [ 52  60  55]]

 [[ 85  91  86]
  [ 87  93  86]
  [ 85  92  84]
  ...
  [ 51  61  58]
  [ 53  60  59]
  [ 52  59  58]]

 ...

 [[162 155 156]
  [161 154 154]
  [162 156 154]
  ...
  [144 145 141]
  [144 145 141]
  [144 145 141]]

 [[162 155 156]
  [161 154 155]
  [162 155 156]
  ...
  [144 145 141]
  [144 145 141]
  [144 145 141]]

 [[166 157 154]
  [166 156 152]
  [168 157 154]
  ...
  [144 145 141]
  [146 147 142]
  [146 147 142]]]
[[[ 73  84  86]
  [ 76  84  86]
  [ 82  86  90]
  ...
  [ 53  61  46]
  [ 53  61  46]
  [ 49  57  43]]

 [[ 79  85  87]
  [ 80  85  87]
  [ 81  85  87]
  ...
  [ 54  60  53]
  [ 53  59  52]
  [ 52  58  51]]

 [[ 80  87  86]
  [ 81  87  86]
  [ 81  85  85]
  ...
  [ 54  60  54]
  [ 54  60  55]
  [ 53  59  53]]

 ...

 [[164 158 154]
  [162 158 153]
  [161 158 152]
  ..

[[[ 73  89  81]
  [ 73  85  82]
  [ 75  84  86]
  ...
  [ 56  60  54]
  [ 58  58  58]
  [ 60  59  60]]

 [[ 72  88  80]
  [ 73  86  80]
  [ 77  86  83]
  ...
  [ 53  61  52]
  [ 53  59  53]
  [ 55  61  56]]

 [[ 72  88  80]
  [ 74  87  80]
  [ 78  88  83]
  ...
  [ 52  60  48]
  [ 54  61  51]
  [ 55  62  52]]

 ...

 [[164 158 154]
  [164 158 153]
  [164 158 152]
  ...
  [140 145 143]
  [140 145 144]
  [140 145 145]]

 [[162 157 151]
  [162 157 151]
  [164 158 152]
  ...
  [139 145 140]
  [139 145 140]
  [140 146 141]]

 [[164 154 148]
  [164 157 150]
  [164 159 150]
  ...
  [138 144 138]
  [139 145 140]
  [140 146 141]]]
[[[ 85  92  84]
  [ 85  90  86]
  [ 84  89  89]
  ...
  [ 55  60  55]
  [ 54  60  55]
  [ 55  61  56]]

 [[ 85  92  84]
  [ 84  91  87]
  [ 82  89  89]
  ...
  [ 54  59  56]
  [ 54  59  58]
  [ 54  59  58]]

 [[ 83  92  84]
  [ 83  92  85]
  [ 82  90  85]
  ...
  [ 53  59  56]
  [ 52  59  58]
  [ 53  60  60]]

 ...

 [[165 159 153]
  [162 157 151]
  [162 157 151]
  ..

[[[ 84  91  93]
  [ 83  90  91]
  [ 82  89  89]
  ...
  [ 58  52  56]
  [ 61  56  59]
  [ 65  60  62]]

 [[ 84  91  91]
  [ 83  90  90]
  [ 82  89  89]
  ...
  [ 57  53  55]
  [ 61  56  59]
  [ 64  58  61]]

 [[ 83  91  86]
  [ 83  91  88]
  [ 83  90  90]
  ...
  [ 58  55  58]
  [ 60  57  60]
  [ 62  59  61]]

 ...

 [[167 158 157]
  [167 158 157]
  [167 158 157]
  ...
  [140 146 141]
  [140 146 141]
  [140 146 141]]

 [[165 158 157]
  [164 158 157]
  [163 158 158]
  ...
  [140 145 143]
  [141 146 146]
  [141 146 146]]

 [[163 157 160]
  [163 157 160]
  [163 157 160]
  ...
  [142 145 144]
  [143 146 146]
  [145 147 147]]]
[[[ 76  91  83]
  [ 74  90  82]
  [ 73  89  81]
  ...
  [ 55  61  50]
  [ 54  61  51]
  [ 56  63  54]]

 [[ 77  90  82]
  [ 76  91  83]
  [ 73  89  81]
  ...
  [ 56  61  49]
  [ 56  61  49]
  [ 57  62  50]]

 [[ 78  89  81]
  [ 77  89  81]
  [ 77  87  80]
  ...
  [ 55  61  51]
  [ 53  60  50]
  [ 53  60  50]]

 ...

 [[164 157 157]
  [162 157 155]
  [160 157 152]
  ..

[[[ 84  87  73]
  [ 85  87  73]
  [ 85  88  74]
  ...
  [ 50  60  54]
  [ 50  61  53]
  [ 52  63  55]]

 [[ 85  88  74]
  [ 84  88  74]
  [ 83  88  74]
  ...
  [ 51  60  54]
  [ 53  62  54]
  [ 54  63  55]]

 [[ 82  90  75]
  [ 81  88  76]
  [ 82  89  79]
  ...
  [ 52  59  53]
  [ 54  60  53]
  [ 55  61  54]]

 ...

 [[166 159 148]
  [166 159 150]
  [166 158 152]
  ...
  [142 143 139]
  [142 143 138]
  [142 143 138]]

 [[167 158 145]
  [167 158 146]
  [167 158 147]
  ...
  [143 143 143]
  [145 143 143]
  [146 143 143]]

 [[167 159 143]
  [164 159 144]
  [161 157 144]
  ...
  [146 145 147]
  [146 144 146]
  [146 142 145]]]
[[[ 80  87  73]
  [ 82  88  71]
  [ 85  89  71]
  ...
  [ 56  57  51]
  [ 60  60  53]
  [ 62  61  54]]

 [[ 83  90  78]
  [ 83  90  75]
  [ 83  89  72]
  ...
  [ 57  57  51]
  [ 60  60  53]
  [ 62  61  54]]

 [[ 84  92  80]
  [ 83  91  76]
  [ 81  89  72]
  ...
  [ 60  59  51]
  [ 59  58  52]
  [ 60  60  53]]

 ...

 [[166 157 154]
  [163 156 152]
  [162 156 152]
  ..

KeyboardInterrupt: 

In [17]:
path = r"E:\DEEP-LEARNING\video-classification\images\swimming"
for img in os.listdir(path):
    h,t = os.path.splitext(img)
    if t in [".jpe"]:
        print(t)